**预测部署作业**

> 在AiStudio中基于训练的检测与识别模型，跑通Paddle Serving部署

In [1]:
!git clone https://gitee.com/paddlepaddle/PaddleOCR

正克隆到 'PaddleOCR'...
remote: Enumerating objects: 31443, done.
remote: Counting objects: 100% (5822/5822), done.
remote: Compressing objects: 100% (2358/2358), done.
remote: Total 31443 (delta 3941), reused 5082 (delta 3373), pack-reused 25621
接收对象中: 100% (31443/31443), 258.72 MiB | 8.81 MiB/s, 完成.
处理 delta 中: 100% (21802/21802), 完成.
检查连接... 完成。


In [2]:
import os
# 修改代码运行的默认目录为 /home/aistudio/PaddleOCR
os.chdir("/home/aistudio/PaddleOCR")
# 安装PaddleOCR第三方依赖
!pip install --upgrade pip --user
!pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.7MB 14.4MB/s eta 0:00:01     |████▋                           | 245kB 14.4MB/s eta 0:00:01
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.1 MB 20.4 MB/s            
     |████████████████████████████████| 13.3 MB 43.4 MB/s            
     |████████████████████████████████| 948 kB 62.0 MB/s            
     |████████████████████████████████| 603 kB 65.8 MB/s            
     |████████████████████████████████| 299 kB 66.5 MB/s            
     |████████████████████████████████| 50 kB 13.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 55.7 MB 46 kB/s              
     |████████████████████████████████| 6.4 MB 62.4 MB/s            
     |████████████████████████████████

In [3]:
#安装Paddle Serving三个安装包：paddle-serving-server、paddle-serving-client 和 paddle-serving-app
!wget https://paddle-serving.bj.bcebos.com/test-dev/whl/paddle_serving_server_gpu-0.7.0.post102-py3-none-any.whl
!pip install paddle_serving_server_gpu-0.7.0.post102-py3-none-any.whl

!wget https://paddle-serving.bj.bcebos.com/test-dev/whl/paddle_serving_client-0.7.0-cp37-none-any.whl
!pip install paddle_serving_client-0.7.0-cp37-none-any.whl

!wget https://paddle-serving.bj.bcebos.com/test-dev/whl/paddle_serving_app-0.7.0-py3-none-any.whl
!pip install paddle_serving_app-0.7.0-py3-none-any.whl

!rm ./*.whl

--2021-12-26 01:59:43--  https://paddle-serving.bj.bcebos.com/test-dev/whl/paddle_serving_server_gpu-0.7.0.post102-py3-none-any.whl
Resolving paddle-serving.bj.bcebos.com (paddle-serving.bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddle-serving.bj.bcebos.com (paddle-serving.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7146257 (6.8M) [application/octet-stream]
Saving to: ‘paddle_serving_server_gpu-0.7.0.post102-py3-none-any.whl’

paddle_serving_serv 100%[===================>]   6.81M  34.6MB/s    in 0.2s    

2021-12-26 01:59:43 (34.6 MB/s) - ‘paddle_serving_server_gpu-0.7.0.post102-py3-none-any.whl’ saved [7146257/7146257]

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Processing ./paddle_serving_server_gpu-0.7.0.post102-py3-none-any.whl
     |████████████████████████████████| 2.5 MB 59.2 MB/s            
     |████████████████████████████████| 44 kB 4.6 MB/

In [4]:
#准备部署模型
os.chdir("/home/aistudio/PaddleOCR/deploy/pdserving/")

# 下载并解压 OCR 文本检测模型
!wget https://paddleocr.bj.bcebos.com/PP-OCRv2/chinese/ch_PP-OCRv2_det_infer.tar -O ch_PP-OCRv2_det_infer.tar && tar -xf ch_PP-OCRv2_det_infer.tar && rm ch_PP-OCRv2_det_infer.tar
# 下载并解压 OCR 文本识别模型
!wget https://paddleocr.bj.bcebos.com/PP-OCRv2/chinese/ch_PP-OCRv2_rec_infer.tar -O ch_PP-OCRv2_rec_infer.tar &&  tar -xf ch_PP-OCRv2_rec_infer.tar && rm ch_PP-OCRv2_rec_infer.tar

--2021-12-26 02:00:17--  https://paddleocr.bj.bcebos.com/PP-OCRv2/chinese/ch_PP-OCRv2_det_infer.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3190272 (3.0M) [application/x-tar]
Saving to: ‘ch_PP-OCRv2_det_infer.tar’

ch_PP-OCRv2_det_inf 100%[===================>]   3.04M  6.80MB/s    in 0.4s    

2021-12-26 02:00:18 (6.80 MB/s) - ‘ch_PP-OCRv2_det_infer.tar’ saved [3190272/3190272]

--2021-12-26 02:00:18--  https://paddleocr.bj.bcebos.com/PP-OCRv2/chinese/ch_PP-OCRv2_rec_infer.tar
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|182.61.200.229|:443... connected.
HTTP request sent, awaiting response... 200 OK

In [5]:
# 转换检测模型
!python -m paddle_serving_client.convert --dirname ./ch_PP-OCRv2_det_infer/ \
                                         --model_filename inference.pdmodel          \
                                         --params_filename inference.pdiparams       \
                                         --serving_server ./ppocrv2_det_serving/ \
                                         --serving_client ./ppocrv2_det_client/

# 转换识别模型
!python -m paddle_serving_client.convert --dirname ./ch_PP-OCRv2_rec_infer/ \
                                         --model_filename inference.pdmodel          \
                                         --params_filename inference.pdiparams       \
                                         --serving_server ./ppocrv2_rec_serving/  \
                                         --serving_client ./ppocrv2_rec_client/

# 查看文件夹
!tree -h *_client *_serving

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
ppocrv2_det_client
├── [ 296]  serving_client_conf.prototxt
└── [  98]  serving_client_conf.stream.prototxt
ppocrv2_rec_client
├── [ 284]  serving_client_conf.prototxt
└── [  93]  serving_client_conf.stream.prototxt
ppocrv2_det_serving
├── [2.2M]  inference.pdiparams
├── [842K]  inference.pdmodel
├── [ 296]  serving_server_conf.prototxt
└── [  98]  serving_server_conf.stream.prototxt
ppocrv2_rec_serving
├── [7.9M]  inference.pdiparams
├── [527K]  inference.pdmodel
├── [ 284]  serving_server_conf.prototxt
└── [  93]  serving_server_con

In [ ]:
#addleOCR/deploy/pdserving/config.yml文件中的两个model_config字段分别修改为ppocrv2_det_serving、ppocrv2_rec_serving，对应模型转换的文件夹
#rpc端口, rpc_port和http_port不允许同时为空。当rpc_port为空且http_port不为空时，会自动将rpc_port设置为http_port+1
rpc_port: 18091

#http端口, rpc_port和http_port不允许同时为空。当rpc_port可用且http_port为空时，不自动生成http_port
http_port: 9998

#worker_num, 最大并发数。当build_dag_each_worker=True时, 框架会创建worker_num个进程，每个进程内构建grpcSever和DAG
##当build_dag_each_worker=False时，框架会设置主线程grpc线程池的max_workers=worker_num
worker_num: 10

#build_dag_each_worker, False，框架在进程内创建一条DAG；True，框架会每个进程内创建多个独立的DAG
build_dag_each_worker: False

dag:
    #op资源类型, True, 为线程模型；False，为进程模型
    is_thread_op: False

    #重试次数
    retry: 10

    #使用性能分析, True，生成Timeline性能数据，对性能有一定影响；False为不使用
    use_profile: True
    
    tracer:
        interval_s: 10
op:
    det:
        #并发数，is_thread_op=True时，为线程并发；否则为进程并发
        concurrency: 8

        #当op配置没有server_endpoints时，从local_service_conf读取本地服务配置
        local_service_conf:
            #client类型，包括brpc, grpc和local_predictor.local_predictor不启动Serving服务，进程内预测
            client_type: local_predictor

            #det模型路径
            model_config: ./ppocrv2_det_serving

            #Fetch结果列表，以client_config中fetch_var的alias_name为准
            fetch_list: ["save_infer_model/scale_0.tmp_1"]

            #计算硬件ID，当devices为""或不写时为CPU预测；当devices为"0", "0,1,2"时为GPU预测，表示使用的GPU卡
            devices: "0"
            
            ir_optim: True
    rec:
        #并发数，is_thread_op=True时，为线程并发；否则为进程并发
        concurrency: 4

        #超时时间, 单位ms
        timeout: -1
 
        #Serving交互重试次数，默认不重试
        retry: 1

        #当op配置没有server_endpoints时，从local_service_conf读取本地服务配置
        local_service_conf:

            #client类型，包括brpc, grpc和local_predictor。local_predictor不启动Serving服务，进程内预测
            client_type: local_predictor

            #rec模型路径
            model_config: ./ppocrv2_rec_serving

            #Fetch结果列表，以client_config中fetch_var的alias_name为准
            fetch_list: ["save_infer_model/scale_0.tmp_1"]  

            #计算硬件ID，当devices为""或不写时为CPU预测；当devices为"0", "0,1,2"时为GPU预测，表示使用的GPU卡
            devices: "0"
            
            ir_optim: True


In [ ]:
#开启新的终端运行下列启动服务的命令.启动服务，运行日志保存在web_serving_log.txt
cd PaddleOCR/deploy/pdserving/
nohup python web_service.py &>web_serving_log.txt &
tailf web_serving_log.txt

启动服务后，查看web_serving_log.txt的截图
![](https://ai-studio-static-online.cdn.bcebos.com/2055394919af4314bf3fd25feb6f819f1a04c853ad4f4b14b4137759f6e87377)


In [6]:
#发送服务请求
!python pipeline_http_client.py

{'err_no': 0, 'err_msg': '', 'key': ['res'], 'value': ["['Mairiedu|er|农', '*|Palais du LOUVRE', '公 LES ARTS DECORATIFS', '＊ Musée du LOUVRE', 'Theatre', 'du PALAIS-ROYAL L&']"], 'tensors': []}
{'err_no': 0, 'err_msg': '', 'key': ['res'], 'value': ["['代号', '项目', '结果', '参考值', '单位', '谷丙转氨酶', '25.6', '0--40', 'ALT', 'U/L', 'TBIL', '总胆红素', '11.2', '<20', 'UMO1/L', '直接胆红素', '3.3', '0--7', 'DBIL', 'UMOL/L', '间接胆红素', '7.9', 'IBIL', '1.5--15', 'UMOL/L', '总蛋白', '58.9V', '60--80', 'TP', 'S/L', '白蛋白', '35.1', '33--55', 'ALB', ':/L', '球蛋白', '23.8', '20--30', 'GLO', '8/L', '白球比', 'A/G', '1.5', '1.5--2.5', 'ALP', '可性磷酉酯', '93', '15--112', 'IU/L', '谷氨酰转肽酸', '<50', '14.3', 'GGT', 'U/L', '谷草转氨酶', 'AST', '16.3', '8--40', 'U/L', '乳酸脱氢酶', 'LDH', '167', '114--240', 'U/L', '腺甘脱氨酶', '12.6', '4--24', 'ADA', 'U/L']"], 'tensors': []}
{'err_no': 0, 'err_msg': '', 'key': ['res'], 'value': ["['订007778', 'PASS', '登机牌', 'BOARDING', '座位号', '序号', 'SERIALNO.', '舱位 CLASS', 'SEAT NO.', '航班 FLIGHT', '日期 DATE', 'W', '03DEC'

In [7]:
#预测性能数据会被自动写入 PipelineServingLogs/pipeline.tracer 文件
!cat PipelineServingLogs/pipeline.tracer

2021-12-26 02:09:10,549 ==================== TRACER ======================
2021-12-26 02:09:10,631 Channel (server worker num[10]):
2021-12-26 02:09:10,632 	chl0(In: ['@DAGExecutor'], Out: ['det']) size[0/0]
2021-12-26 02:09:10,633 	chl1(In: ['det'], Out: ['rec']) size[0/0]
2021-12-26 02:09:10,633 	chl2(In: ['rec'], Out: ['@DAGExecutor']) size[0/0]
2021-12-26 02:09:20,643 ==================== TRACER ======================
2021-12-26 02:09:20,644 Channel (server worker num[10]):
2021-12-26 02:09:20,644 	chl0(In: ['@DAGExecutor'], Out: ['det']) size[0/0]
2021-12-26 02:09:20,644 	chl1(In: ['det'], Out: ['rec']) size[0/0]
2021-12-26 02:09:20,645 	chl2(In: ['rec'], Out: ['@DAGExecutor']) size[0/0]
2021-12-26 02:09:30,655 ==================== TRACER ======================
2021-12-26 02:09:30,656 Channel (server worker num[10]):
2021-12-26 02:09:30,656 	chl0(In: ['@DAGExecutor'], Out: ['det']) size[0/0]
2021-12-26 02:09:30,656 	chl1(In: ['det'], Out: ['rec']) size[0/0]
2021-12-26 02:09:30,657

### 提交说明
提交格式为notebook项目，具体提交步骤参考[预习作业](https://aistudio.baidu.com/aistudio/projectdetail/3259472)部分

项目中需提供**操作过程截图**